# Phase 3: Template-to-Template Evaluation

This notebook contains results evaluating the correspondence between RheMAP templates (D99, INIA19, macaqueMNI, NMTv1.3, Yerkes19).

In [1]:
# initialize libraries
library(plyr)
library(digest)
library(reshape2)
library(ggplot2)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


In [2]:
# useful functions

# calculate the distance between two sets of coordinates
dist3D <- function(coord1, coord2) { # vector X,Y,Z
        xdist <- coord1[1] - coord2[1] # could also write as coord1$X, etc.
        ydist <- coord1[2] - coord2[2]
        zdist <- coord1[3] - coord2[3]
        euclidean_dist <- sqrt(xdist^2+ydist^2+zdist^2)
        return(as.numeric(unlist(euclidean_dist)))
}

# calculate the pairwise distance between an array of 3D coordinates
pairwise_dist3D <- function(temp_coords) { # labeled X,Y,Z
        N <- length(temp_coords$X)
        dist_vec <- rep(0,N) # create vector
        sum_dist <- 0 # initialize to zero
        count <- 0
        for (i in 1:(N-1)) {
                for (j in (i+1):N) {
                        if (i != j) {
                                count <- count + 1
                                first_coord <- temp_coords[i,]
                                second_coord <- temp_coords[j,]
                                curr_dist <- dist3D(first_coord, second_coord)
                                sum_dist <- sum_dist + curr_dist
                                dist_vec[count] <- curr_dist
                        }
                }
        }
        return(c(as.numeric(unlist(mean(dist_vec))),as.numeric(unlist(sd(dist_vec)))))
}

In [3]:
# initialize variables and load in raw fcsv data into df_raters
setwd('~/Documents/GitHub/afids-macaca/data/PHASE3_RheMAP_transformed/')

df_afid <- read.table('~/Documents/GitHub/afids-macaca/etc/afids.csv', sep=",", header=TRUE)

df_regs <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),
                        moving_template=factor(),fixed_template=factor(),xfm=factor(),
                        name=character(),description=character(),stringsAsFactors = FALSE)
csv_files <- list.files(".", "*.fcsv")

for (i in 1:length(csv_files)) {
    curr_split <- unlist(strsplit(csv_files[i],"_"))
    if (length(curr_split)>1) { # extract name and session data
        moving_template <- curr_split[1]
        fixed_template <- curr_split[5]
        xfm <- as.character(unlist(strsplit(curr_split[6],"[.]"))[1])
    }
    curr_reg <- read.table(csv_files[i], header=FALSE, sep=",")
    df_reg <- data.frame(fid = 1:length(curr_reg$V1))

    df_reg <- cbind(df_reg,X=curr_reg[2],Y=curr_reg[3],Z=curr_reg[4],moving_template=moving_template,
                    fixed_template=fixed_template,
                    xfm=xfm,name=curr_reg[12],
                    description=curr_reg[13])
  
    df_reg <- rename(df_reg, c("V2"="X","V3"="Y","V4"="Z","V12"="name","V13"="description"))
    df_regs <- rbind(df_regs,df_reg)
}

levels(df_regs$xfm) <- c('nlin','lin') # rename levels
head(df_regs)


fid,X,Y,Z,moving_template,fixed_template,xfm,name,description
1,-0.269,0.416,0.187,d99,inia19,nlin,1,AC
2,-0.160,-12.484,-1.242,d99,inia19,nlin,2,PC
3,-0.238,-19.890,-3.171,d99,inia19,nlin,3,infracollicular sulcus
4,-0.478,-10.446,-10.850,d99,inia19,nlin,4,PMJ
5,-0.359,-8.024,-6.561,d99,inia19,nlin,5,superior interpeduncular fossa
6,6.844,-13.664,-5.708,d99,inia19,nlin,6,R superior LMS


In [4]:
# Start with templates --> D99

# load D99
df_D99_mean <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),stringsAsFactors = FALSE)
D99_file <- "~/Documents/GitHub/afids-macaca/data/PHASE1_output_afid/d99_MEAN.fcsv"

curr_template <- read.table(D99_file, header=FALSE, sep=",")
df_template <- data.frame(fid = 1:length(curr_template$V1))
df_template <- cbind(df_template,X=curr_template[2],Y=curr_template[3],Z=curr_template[4])
df_template <- rename(df_template, c("V2"="X","V3"="Y","V4"="Z"))
df_D99_mean <- df_template

df_D99_results <- subset(df_regs, fixed_template == "d99")
length_df <- length(df_D99_results$fid)
df_D99_results$dist <- rep(0, length_df)

for (val in 1:(length_df-1))
{
    curr_coord <- df_D99_results[val, 2:4]
    curr_fid <- df_D99_results[val,]$fid
    truth_coord <- df_D99_mean[as.numeric(curr_fid),2:4]
    df_D99_results[val,]$dist <- dist3D(curr_coord, truth_coord)
}
#df_D99_results
#subset(df_D99_results, moving_template == 'yerkes19' & xfm == 'nlin')

ddply(df_D99_results, .(moving_template,xfm), summarize, mean_dist = mean(dist), sd_dist = sd(dist))

moving_template,xfm,mean_dist,sd_dist
inia19,nlin,0.8389328,0.4942276
inia19,lin,1.0687900,0.5656165
macaqueMNI,nlin,0.6123858,0.2871578
macaqueMNI,lin,0.8659222,0.4908373
nmtv1.3,nlin,0.5388126,0.4815783
nmtv1.3,lin,0.9604349,0.4719488
yerkes19,nlin,0.7265688,0.4242849
yerkes19,lin,1.4381401,0.5926002


In [5]:
sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Catalina 10.15.4

Matrix products: default
BLAS/LAPACK: /Users/jclau/anaconda3/envs/r-tutorial/lib/R/lib/libRblas.dylib

locale:
[1] en_CA.UTF-8/UTF-8/en_CA.UTF-8/C/en_CA.UTF-8/en_CA.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] ggplot2_3.1.1  reshape2_1.4.3 digest_0.6.18  plyr_1.8.4    

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.1       magrittr_1.5     tidyselect_0.2.5 munsell_0.5.0   
 [5] uuid_0.1-2       colorspace_1.4-1 R6_2.4.0         rlang_0.3.4     
 [9] dplyr_0.8.0.1    stringr_1.4.0    tools_3.6.1      grid_3.6.1      
[13] gtable_0.3.0     withr_2.1.2      htmltools_0.3.6  assertthat_0.2.1
[17] lazyeval_0.2.2   tibble_2.1.1     crayon_1.3.4     IRdisplay_0.7.0 
[21] purrr_0.3.2      repr_0.19.2      base64enc_0.1-3  IRkernel_0.8.15 
[25] glue_1.3.1       evaluate_0.13    pbdZMQ